# Neurosynth decoding

###  Wir machen uns eine Liste mit allen unseren Hirnbildern

In [1]:
import os

In [2]:
imgList = ['../meanTraining/%s'%x for x in os.listdir('../meanTraining/')]; imgList.sort()

### Whole-Brain Maske

Hier benutzen wir eine grobe Maske mit 4mm Auflösung, weil die Berechnungen sonst zu lange dauern

In [3]:
from nilearn import image, plotting, input_data 

In [4]:
import pickle

Die Maske habe ich vorbereitet, die laden wir als hinterlegtes Objekt

In [5]:
my_masker = pickle.load( open( "../pickels/my4mm_masker.p", "rb" ) )

In [6]:
my_masker

NiftiMasker(detrend=False, high_pass=None, low_pass=None, mask_args=None,
      mask_img='../rois/gmMap4mm.nii.gz', mask_strategy='background',
      memory=Memory(cachedir=None), memory_level=1, sample_mask=None,
      sessions=None, smoothing_fwhm=None, standardize=False, t_r=None,
      target_affine=None, target_shape=None, verbose=0)

In [7]:
plotting.plot_roi(my_masker.mask_img_);

### Daten extrahieren

In [8]:
from sklearn import preprocessing

In [9]:
def extractMaps(fileName,my_masker):
    thisData = my_masker.transform(fileName)[-1]
    scaleData = preprocessing.scale(thisData)
    return scaleData

Um das Ganze erstmal beispielhaft durchzugehen, nehmen wir uns einen einzelnen Block aus unseren Daten

In [10]:
thisMap = imgList[-1]

In [11]:
thisMap

'../meanTraining/meanCond_words.nii.gz'

In [12]:
scaleMap = extractMaps(thisMap,my_masker)

AttributeError: 'NiftiMasker' object has no attribute '_shelving'

In [ ]:
scaleMap

### wir laden ca. 600 Karten aus Neurosynth, die ich vorher ausgewählt habe  

Die Einschlusskriterien waren, dass die Maske nicht leer sein darf (Stichwörter wie "magnetic" sind so unspefifisch, dass sie keine Voxel enthalten; außerdem gibt es sehr viele Karten, die zwar Voxel enthalten, aber keinem inhaltlich interpertierbaren psychologischen Prozess zugeordnet werden können (z.B. white matter, young, old, patient, healthy).

In [ ]:
import pandas as pd

In [ ]:
nsData = pd.read_csv('../arrays/ns_4mm_database.csv',index_col=[0,1])

In [ ]:
nsData.tail()

In [ ]:
clusterNames =  pickle.load( open( "../pickels/clusterDict.p", "rb" ) )

In [ ]:
clusterNames

## Korrelation eines Blocks unserer Daten mit allen 602 Karten

In [ ]:
import numpy as np

In [ ]:
thisCorr = np.corrcoef(scaleMap,nsData)[0,:][1:]

In [ ]:
thisCorrDf = pd.DataFrame(thisCorr,index=nsData.index,columns=['corr']).T

In [ ]:
thisCorrDf

### Wir sortieren nach Stärke der Korrelation

Sonst müssten wir die 602 Korrelationen einzeln durchgehen um bedeutsame Zusammenhänge zu finden.

In [ ]:
def getTop(corrDf):
    sortDf = corrDf.copy()
    sortDf = sortDf.T.sort_values(by=sortDf.index[-1],ascending=False)
    topDf = pd.concat([sortDf[:5],sortDf[-5:]],axis=0)
    topDf.columns = ['correlation']
    topDf = topDf.round(2)
    return topDf

In [ ]:
topDf = getTop(thisCorrDf)

Beispiel: Die fünf höchstn und die fünf niedrigsten Korrelationen des gewählten Blocks

In [ ]:
topDf

## Die 602 Karten in Neurosynth nach Ähnlichkeit gruppieren

Wir haben gesagt, dass die 602 Karten in Cluster eingeteilt sind. Können wir das irgendwie darstellen? Ja, indem wir die 18744 Dimensionen (jeder Voxel ist eine Dimension) auf 2 Dimensionen projizieren. Dazu verwenden wir [Multidimensionale Skalierung](http://scikit-learn.org/stable/modules/manifold.html).

In [ ]:
# hier verwenden wir Daten die ich vorbereitet habe
dissDf = pd.read_csv('../arrays/dissDf.csv',index_col=[0])
mdsPositions = np.array(pd.read_csv('../arrays/mdsDf.csv',index_col=[0]))
mdsDf = pd.DataFrame(mdsPositions,index=dissDf.index)

In [ ]:
# plotting
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
%matplotlib inline

Acht Fraben für die acht Cluster:

In [ ]:
myPalette = sns.color_palette('Set1',n_colors=8)
sns.palplot(myPalette)

## Brain Space

In [ ]:
# laden von vorbereiteten Sachen
mdsDf = pd.read_csv('../arrays/mdsDf.csv',index_col=[0])
kDf = pd.read_csv('../arrays/kDf.csv',index_col=[0])

Abbildung erstellen

In [ ]:
def findNeighbors(mdsDf,p,added,notCloserThan=50):
    
    # coordinates of this keyword
    thisDf = mdsDf.loc[p]
    
    # coordinates of all other keywords
    otherDf = mdsDf.drop(p)
    
    # lenghts of adjacent and opposite
    diffDf = abs(thisDf-otherDf)
    
    # lengths of hypoteneuse
    distanceDf = np.sqrt(diffDf**2).sum(axis=1)
    
    # check if there are close distances
    closeEncounters = distanceDf[distanceDf<notCloserThan].index
    
    # check if the close ones have already been labelled
    for entry in closeEncounters:
        if entry in added:
            return True
    
    return False

Einstellungen für Abbildung

In [ ]:
sns.set_style('white')
sns.set_context('poster')

### der große Ball

In [ ]:
def plotSpace(mdsDf,kDf,clusterNames,myPalette,closest,ax):

    kPredictions = list(kDf['n'])
    # loop both trough the positions and the predictions
    for p,l in zip(mdsDf.index,kPredictions):
        # show predictions from raw data on mds scaled data, the predictions are indicated by the color
        ax.plot( mdsDf.ix[p]['0'],
                     mdsDf.ix[p]['1'],
                     'o',color=myPalette[l],
                      markersize=12,alpha=0.8
               )

    added = []
    mdsDf = mdsDf.sort_values(by='1')
    # sorting by the x-dimension will fill the labels from right to left side
    mdsDf = mdsDf.sort_values(by='0',ascending=False)
    for p,x,y in zip(mdsDf.index,mdsDf['0'], mdsDf['1']):
        l = kDf.loc[p]
        if not findNeighbors(mdsDf,p,added,notCloserThan=closest):
            ax.annotate(p, xy = (x, y),fontsize=16,alpha=0.8)
            added.append(p)
            
    # dummy plot for labelling
    for l in np.unique(kDf['n']):
        ax.plot([],'o',color=myPalette[l],label=clusterNames[str(l)])
    
    # show the plot
    sns.despine(left=True,bottom=True)
    ax.set_xticks([]);ax.set_yticks([])
    ax.legend(loc='lower left',bbox_to_anchor=(0.,0))
    return ax

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,16))
ax = plotSpace(mdsDf,kDf,clusterNames,myPalette,0,ax)
#plt.savefig('../figs/nsBallSparse.png',dpi=300)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,16))
ax = plotSpace(mdsDf,kDf,clusterNames,myPalette,50,ax)
plt.savefig('../figs/nsBallSparse.png',dpi=300)
plt.show()

## der Ball - Teil 2

In [ ]:
# Skalierung der Korrelationen von 0 bis 1
def makeMinMax(corrDf):
    minMaxDf = pd.DataFrame( preprocessing.minmax_scale(corrDf,axis=1),
                            index=corrDf.index,
                            columns=corrDf.columns )
    return minMaxDf.T

In [ ]:
minMaxDf = makeMinMax(thisCorrDf)

In [ ]:
thisCorrDf

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(12,5))
sns.distplot(thisCorrDf.T,ax=ax1)
sns.distplot(minMaxDf,ax=ax2)
sns.despine()
plt.show()

Abbildung machen

In [ ]:
def plotFullCorr(mdsDf,kDf,minMaxDf,myPalette,clusterNames,ax):

    kPredictions = list(kDf['n'])
    # loop both trough the positions and the predictions
    for p,l in zip(mdsDf.index,kPredictions):
        # show predictions from raw data on mds scaled data, the predictions are indicated by the color
        ax.plot( mdsDf.ix[p]['0'],
                     mdsDf.ix[p]['1'],
                     'o',color=myPalette[l],
                      markersize=minMaxDf.ix[l].ix[p]**3*50,
                      alpha=0.7
               )

    for p,l,x,y in zip(mdsDf.index,kPredictions, mdsDf['0'], mdsDf['1']):
        ax.annotate(p, xy = (x, y),
                    fontsize=minMaxDf.ix[l].ix[p]**3*50,
                    alpha=minMaxDf.ix[l].ix[p]**10)

            
    # dummy plot for labelling
    for l in np.unique(kDf['n']):
        ax.plot([],'o',color=myPalette[l],label=clusterNames[str(l)])
    
    # show the plot
    sns.despine(left=True,bottom=True)
    ax.set_xticks([]);ax.set_yticks([])
    #ax.legend(loc='lower left',bbox_to_anchor=(1.5,0))
    return ax

In [ ]:
def plotMeanCorr(mdsDf,kDf,minMaxDf,myPalette,clusterNames,closest,ax):

    # loop both trough the positions and the predictions
    for p in mdsDf.index:
        l = kDf['n'].loc[p]
        # show predictions from raw data on mds scaled data, the predictions are indicated by the color
        ax.plot( mdsDf.ix[p]['0'],
                     mdsDf.ix[p]['1'],
                     'o',color=myPalette[l],
                      markersize=minMaxDf.ix[l].ix[p]**3*50,
                      alpha=0.7
               )

    # to not omit the most important keywords, we move through the list by the order in minMaxDf
    sortedIndex = minMaxDf.sort_values('corr',ascending=False).index.labels[1]
    sortedNames = [minMaxDf.index.levels[1][x] for x in sortedIndex ]
    print sortedNames[:10]
    added = []
    for p in sortedNames:
        l = kDf['n'].loc[p]
        x = mdsDf['0'].loc[p]
        y = mdsDf['1'].loc[p]

        if not findNeighbors(mdsDf,p,added,notCloserThan=closest):

            thisVal = minMaxDf.ix[l].ix[p].values[-1]
            
            ax.annotate(p, xy = (x, y),
                    fontsize=thisVal**3*50,
                    alpha=thisVal**5)
            added.append(p)

    # dummy plot for labelling
    for l in np.unique(kDf['n']):
        ax.plot([],'o',color=myPalette[l],label=clusterNames[str(l)])
    
    # show the plot
    sns.despine(left=True,bottom=True)
    ax.set_xticks([]);ax.set_yticks([])
    #ax.legend(loc='lower left',bbox_to_anchor=(1.5,0))
    return ax

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
plotMeanCorr(mdsDf,kDf,minMaxDf,myPalette,clusterNames,0,ax)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
plotMeanCorr(mdsDf,kDf,minMaxDf,myPalette,clusterNames,50,ax)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
plotMeanCorr(mdsDf,kDf,minMaxDf,myPalette,clusterNames,100,ax)
plt.show()

In [ ]:
def makeMeanMindSpace(thisMap,closest=100,
                  my_masker=my_masker,nsData=nsData,mdsDf=mdsDf,kDf=kDf,
                  myPalette=myPalette,clusterNames=clusterNames):

    scaleMap = extractMaps(thisMap,my_masker)

    thisCorr = np.corrcoef(scaleMap,nsData)[0,:][1:]
    
    thisCorrDf = pd.DataFrame(thisCorr,index=nsData.index,columns=['corr']).T
    
    minMaxDf = makeMinMax(thisCorrDf)

    fig,ax = plt.subplots(1,1,figsize=(8,8))
    
    plotMeanCorr(mdsDf,kDf,minMaxDf,myPalette,clusterNames,closest,ax)
    
    fullName = thisMap.split('/')[-1].split('.')[0]
    nam = fullName.split('_')[-1]
    plt.title(nam,fontsize=32)
    
    plt.savefig('../nsfigs/synth_%s.png'%fullName,bbox_inches='tight',dpi=300)
    plt.show()

In [ ]:
for im in ['../meanTraining/%s'%x for x in os.listdir('../meanTraining/')]:
    makeMeanMindSpace(im)

### For single blocks

In [ ]:
def plotCorr(mdsDf,kDf,minMaxDf,myPalette,clusterNames,ax):

    # loop both trough the positions and the predictions
    for p in mdsDf.index:
        l = kDf['n'].loc[p]
        # show predictions from raw data on mds scaled data, the predictions are indicated by the color
        ax.plot( mdsDf.ix[p]['0'],
                     mdsDf.ix[p]['1'],
                     'o',color=myPalette[l],
                      markersize=minMaxDf.ix[l].ix[p]**3*50,
                      alpha=0.7
               )

    # to not omit the most important keywords, we move through the list by the order in minMaxDf
    sortedIndex = minMaxDf.sort_values('corr',ascending=False).index.labels[1]
    sortedNames = [minMaxDf.index.levels[1][x] for x in sortedIndex ]
        
    added = []
    counter = 5
    for p in sortedNames:
        l = kDf['n'].loc[p]
        x = mdsDf['0'].loc[p]
        y = mdsDf['1'].loc[p]

        if counter > 0:
            if not findNeighbors(mdsDf,p,added,notCloserThan=130):

                ax.annotate(p, xy = (x, y),
                        fontsize=minMaxDf.ix[l].ix[p]**3*70,
                        alpha=minMaxDf.ix[l].ix[p]**30*5)
                added.append(p)
                
                counter-=1
            
    # dummy plot for labelling
    for l in np.unique(kDf['n']):
        ax.plot([],'o',color=myPalette[l],label=clusterNames[str(l)])
    
    # show the plot
    sns.despine(left=True,bottom=True)
    ax.set_xticks([]);ax.set_yticks([])
    #ax.legend(loc='lower left',bbox_to_anchor=(1.5,0))
    return ax

In [ ]:
def makeMindSpace(thisMap,
                  my_masker=my_masker,nsData=nsData,mdsDf=mdsDf,kDf=kDf,
                  myPalette=myPalette,clusterNames=clusterNames):

    scaleMap = extractMaps(thisMap,my_masker)

    thisCorr = np.corrcoef(scaleMap,nsData)[0,:][1:]
    
    thisCorrDf = pd.DataFrame(thisCorr,index=nsData.index,columns=['corr']).T
    
    minMaxDf = makeMinMax(thisCorrDf)

    fig,ax = plt.subplots(1,1,figsize=(8,8))
    
    plotCorr(mdsDf,kDf,minMaxDf,myPalette,clusterNames,ax)
    
    fullName = thisMap.split('/')[-1].split('.')[0]
    nam = fullName.split('_')[-1]
    num = fullName.split('_')[1]
    plt.title('%s %s' % (num,nam),fontsize=72,y=1.04)
    
    plt.savefig('../nsfigs/synth_%s.png'%fullName,bbox_inches='tight',dpi=300)
    plt.show()

In [ ]:
def sortBlocks(blocks):
    d = {}
    for i in blocks:
        num = i.split('/')[-1].split('.')[0]
        d[num] = i
    sortRunDf = pd.DataFrame(d,index=['filename']).T
    sortRunDf.sort_index(inplace=True)
    return sortRunDf

In [ ]:
imgList = ['../test/%s'%x for x in os.listdir('../test/')]; imgList.sort()

In [ ]:
sortRunDf = sortBlocks(imgList)

In [ ]:
sortRunDf

In [ ]:
for i in sortRunDf.index:
    thisBlock = sortRunDf.ix[i]['filename']
    makeMindSpace(thisBlock)

In [ ]:
from PIL import Image

In [ ]:
ballList = ['../nsfigs/%s'%x for x in os.listdir('../nsfigs/') if x.startswith('synth') and '_00' not in x and 'meanCond' not in x]
ballList.sort()

In [ ]:
ballList

In [ ]:
a = []
for i in ballList:
    im = Image.open(i)
    thisAspect = im.getbbox()
    a.append(thisAspect)
    
whereMax = pd.DataFrame(a).idxmax(axis=0).max()
maxSize = tuple( pd.DataFrame(a).loc[whereMax]+2 )

In [ ]:
fig = plt.figure(figsize=(16,14))
for i,im in enumerate(ballList):
    ax = plt.subplot(5,5,i+1)
    
    mask = Image.new('RGBA', maxSize[2:],color=(255,255,255))
    mask.paste(Image.open(im),(0,0))
    ax.imshow( mask )
    ax.set_xticks([]);ax.set_yticks([])
    sns.despine(left=True,bottom=True)
plt.savefig('../nsfigs/nsSpacesTest.png',dpi=600,bbox_inches='tight')
plt.show()

### for the secret blocks

In [ ]:
for im in ['../outofsample/%s'%x for x in os.listdir('../outofsample/')]:
    makeMeanMindSpace(im)